# Creating Custom Tools for LLM Agents

This notebook showcases different ways to create the custom tools for LLM Agents. Here, Local LLM agents are run on the iGPU of Intel® Core™ Ultra Processors. For this, Llamacpp GPU backend is setup and the agent created using the local LLM model. 

#### Table of Contents
1. Initial setup
      - Download the huggingface models
      - Select Local LLM Model
      - Initialize LlamaCpp Model
2. Create the agent
      - Tools
      - Prompt Template
      - Agent
3. Run the agent
      - Agent Executor
      - Custom tool queries

### 1. Initial setup

#### Download the huggingface models
This step is optional if you've already dowloaded the models in the terminal using huggingface-cli as outlined in the README.md.

In [1]:
from huggingface_hub import hf_hub_download

hf_hub_download(
    repo_id="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF",
    filename="Meta-Llama-3.1-8B-Instruct-Q4_K_S.gguf",
    local_dir="./models"
)

'models\\Meta-Llama-3.1-8B-Instruct-Q4_K_S.gguf'

In [2]:
hf_hub_download(
    repo_id="bartowski/Qwen2.5-7B-Instruct-GGUF",
    filename="Qwen2.5-7B-Instruct-Q4_K_S.gguf",
    local_dir="./models"
)

'models\\Qwen2.5-7B-Instruct-Q4_K_S.gguf'

#### Select Local LLM Model
Select a Local Large language model from the dropdown list.

In [3]:
import os
import ipywidgets as widgets
from IPython.display import display

def find_gguf_files(directory):
    """
    This function can be used to find the GGUF models present in the directory.
    If the filename ends with .gguf then the new model name will be appended to gguf_files list.

    Raises:
		Exception: If there is any error during finding the GGUF models, an error is displayed.
    
    """
    try:
        gguf_files = []
        for file in os.listdir(directory):
            if file.endswith('.gguf'):
                gguf_files.append(file)

        return gguf_files
            
    except Exception as e:
        print(f"Error: Finding the GGUF models: {str(e)}")

gguf_files = find_gguf_files("./models")


"""
Download the models under `./models` folder to get the model name in the widgets dropdown options and for the model usage.
Select a local LLM model from the dropdown list.
If not selected explicitly from the dropdown list, as mentioned in the value Meta-Llama-3.1-8B-Instruct-Q4_K_S.gguf selected automatically. 

Raises:
		Exception: If there is any error during the model an error is displayed.
"""

if len(gguf_files) == 0:
    print(f"No GGUF model was found in this directory.")

if len(gguf_files):
    try:
        selected_model = widgets.Dropdown(
            options=gguf_files,
            value='Meta-Llama-3.1-8B-Instruct-Q4_K_S.gguf',
            description='Model:',
            disabled=False
        )
    
        display(selected_model) 
    except Exception as e:
        print(f"Error: Model not selected:{str(e)}")

Dropdown(description='Model:', options=('Meta-Llama-3.1-8B-Instruct-Q4_K_S.gguf', 'Qwen2.5-7B-Instruct-Q4_K_S.…

#### Initialize LlamaCpp Model

LlamaCpp is a high-performance C++ backend designed for efficient inference and deployment of LLM models. The python wrapper for this is Llamacpp-Python which integrates these optimizations into Python, allowing developers to deploy LLaMA models efficiently with enhanced language understanding and generation capabilities.

**Note**: Please make sure that [LlamaCpp installation process](https://github.com/seshasrinivaspendyala/AI-Travel-Agent/blob/main/README.md#setting-up-environment-and-llamacpp-python-gpu-backend) is completed before proceeding to the next step as outlined in the README.md.

#### Setting up environment and LlamaCPP-python GPU backend

Open a new terminal and perform the following steps:

1. **Create and activate the conda environment**\
    `conda create -n gpu_llmsycl python=3.11 -y`\
    `conda activate gpu_llmsycl`
2. **Initialize oneAPI environment**\
    `@call "C:\Program Files (x86)\Intel\oneAPI\setvars.bat" intel64 --force`
3. **Set the environment variables**\
    `set CMAKE_GENERATOR=Ninja`\
    `set CMAKE_C_COMPILER=cl`\
    `set CMAKE_CXX_COMPILER=icx`\
    `set CXX=icx`\
    `set CC=cl`\
    `set CMAKE_ARGS="-DGGML_SYCL=ON -DGGML_SYCL_F16=ON -DCMAKE_CXX_COMPILER=icx -DCMAKE_C_COMPILER=cl"`
4. **Install Llamacpp-Python bindings**\
    `pip install llama-cpp-python==0.3.1 -U --force --no-cache-dir --verbose`

In [4]:
""" 
Below shows how to load a local LLM using Llamacpp-python GPU backend for SYCL.
"""

from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

"""
Download and copy the models under `./models` folder. Create and initialize the LlamaCpp with the selected model. Model and hyperparameters can be changed based on the end user requirements. 
Here we are using Meta Llama 3.1(Q4_K_S) model which is configured using some hyperparameters, such as GPU Layers to be offloaded on 32 layers for GPU-accelerated inference, Context Length of 4096 tokens.
Temperature set as 0 for deterministic output, Top-P Sampling as 0.95 for controlled randomness and Batch Size as 512 for parallel processing

Raises:
    Exception: If there is any error during model loading an error is displayed. 
"""
try:
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    llm = LlamaCpp(
        model_path="models/" + selected_model.value,   # Path to the Llama model file
        n_gpu_layers=-1,                               # Number of layers to be loaded into gpu memory (default: 0)
        seed=512,                                      # Random number generator (RNG) seed (default: -1, -1 = random seed)
        n_ctx=4096,                                    # Token context window (default: 512)
        f16_kv=True,                                   # Use half-precision for key/value cache (default: True)
        callback_manager=callback_manager,             # Pass the callback manager for output handling
        verbose=False,                                 # Print verbose output (default: True)
        temperature=0,                                 # Temperature controls the randomness of generated text during sampling (default: 0.8)
        top_p=0.95,                                    # Top-p sampling picks the next token from top choices with a combined probability ≥ p (default: 0.95)
        n_batch=512,                                   # Number of tokens to process in parallel (default: 8)
    )
    
    llm.client.verbose = False                         # Print verbose state information (default: True)
except Exception as e:
    print(f"Model loading error: {str(e)}")

### 2. Create the agent

#### Tools

Let us create the following tools.

1. Weather Tool: Fetches weather information.
2. Fahrenheit Conversion: Converts Temperature from Celsius to Fahrenheit.
3. Wikipedia Tool: The [Wikipedia tool](https://python.langchain.com/docs/integrations/tools/wikipedia/) is used to allow the agent to access web information for answering user queries.

For more information refer:
- [Custom tools](https://python.langchain.com/docs/how_to/custom_tools/)
- [Langchain - Wikipedia](https://python.langchain.com/api_reference/community/utilities/langchain_community.utilities.wikipedia.WikipediaAPIWrapper.html)


In [5]:
from langchain_core.tools import tool

tools = []

In [6]:
# Custom weather tool

@tool
def weather(
    city_name: str,
) -> str:
    """
    Fetch the current weather information for a given city.

    Args:
        city_name (str): The name of the city to fetch weather for.

    Returns:
        str: A string representation of the weather data, including temperature, 
             humidity, description, and observation time.

    Raises:
        Exception: If an error occurs during the request or data processing.
    """

    if not isinstance(city_name, str):
        raise TypeError("City name must be a string")

    key_selection = {
        "current_condition": [
            "temp_C",
            "FeelsLikeC",
            "humidity",
            "weatherDesc",
            "observation_time",
        ],
    }
    try:
        import requests

        resp = requests.get(f"https://wttr.in/{city_name}?format=j1")
        resp.raise_for_status()
        resp = resp.json()
        ret = {k: {_v: resp[k][0][_v] for _v in v} for k, v in key_selection.items()}
    
        return str(ret)
    except Exception as e:
        print(f"Unexpected error occured while fetching weather information: {str(e)}")
    
tools.append(weather)

In [7]:
@tool
def fahrenheit_conversion(celsius: int) -> float:
    """
    Convert the temperature from Celsius to Fahrenheit.

    This function takes a temperature in Celsius and converts it to Fahrenheit
    using the formula: F = (C * 1.8) + 32.

    Args:
        celsius (int): The temperature in Celsius to be converted.

    Returns:
        float: The equivalent temperature in Fahrenheit.

    Raises:
        Exception: If there is any error during the conversion, an error is displayed.
    """
    try:
        f = (celsius * 1.8) + 32
        return f
    except Exception as e:
        print(f"Error occured while converting celsius to fahrenheit: {str(e)}")
    
tools.append(fahrenheit_conversion)

In [8]:
from langchain.agents import Tool
from langchain.utilities import WikipediaAPIWrapper
"""
Using Langchain WikipediaAPIWrapper Tool which uses the Wikipedia API to conduct searches and fetch page summaries.

Raises:
    Exception: If there is any error during the loading of the WikipediaAPIWrapper tool, an error is displayed.
"""
try:
    wikipedia = WikipediaAPIWrapper()
    wikipedia_tool = Tool(
        name='wikipedia',
        func= wikipedia.run,
        description="Useful for when you need to look up a topic, country or person on wikipedia"
    )
    tools.append(wikipedia_tool)
except Exception as e:
        print(f"Error occured while loading wikipedia tool: {str(e)}")

#### Prompt template

In [9]:
"""
The following Prompt template is for the Structured chat agent and is customised to handle the user queries.
"""

PREFIX = """[INST]Respond to the human as helpfully and accurately as possible. You have access to the following tools:"""

FORMAT_INSTRUCTIONS = """Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Use the weather tool for all weather and climate related queries. Give the requested information to the human.
Use the fahrenheit_conversion tool to convert celsius numerical value to fahrenheit numerical value. For Example:
    Question: What is the Fahrenheit temperature of 10 degrees celsius?
    Answer: 10.0 degrees celsius is equivalent to 50.0 degrees Fahrenheit.
    Question: What is the Fahrenheit temperature of 20 degrees celsius?
    Answer: 20.0 degrees celsius is equivalent to 68.0 degrees Fahrenheit.

Use wikipedia_tool to look up for a topic, country or person on wikipedia.

Provide the complete Final Answer. Do not truncate the response.
Always consider the user preferences, any specific requirements mentioned in their query.

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{{{
  "action": TOOL_NAME,
  "action_input":  the input to the action
}}}}
```

Follow this format:

Question: input question to answer
Thought: you should always think about what to do. consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: the result of the action
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond. I now know the final answer.
Action:
```
{{{{
  "action": "Final Answer",
  "action_input": "Provide the detailed Final Answer to the human."
}}}}
```[/INST]"""

SUFFIX = """Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation:.
Thought:[INST]"""

HUMAN_MESSAGE_TEMPLATE = "{input}\n\n{agent_scratchpad}"

#### Agent
[**StructuredChatAgent**](https://api.python.langchain.com/en/latest/agents/langchain.agents.structured_chat.base.StructuredChatAgent.html): A specialized agent is capable of using multi-input tools and designed to handle structured conversations using the specified language model and tools.



In [10]:
"""
Creating and initialising a structured chat agent using the LLM and defined tools.

    llm : LLM to be used
    
    tools : list
        List of tools to use
        
    PREFIX : str
        Prefix string prepended to the agent's input. 
        
    SUFFIX : str
        Suffix string appended to the agent's input. 

    HUMAN_MESSAGE_TEMPLATE : str
        Template defining the structure of human messages.

    FORMAT_INSTRUCTIONS : str
        Format instructions for the agent

    Raises:
		Exception: If there is any error during the agent creation, an error is displayed

"""
from langchain.agents import StructuredChatAgent

try:
    agent = StructuredChatAgent.from_llm_and_tools(
        llm,                                           # LLM to use                            
        tools,                                         # Tools available for the agent    
        prefix=PREFIX,                                 # Prefix to prepend to the input
        suffix=SUFFIX,                                 # Suffix to append to the input
        human_message_template=HUMAN_MESSAGE_TEMPLATE, # Template for human messages
        format_instructions=FORMAT_INSTRUCTIONS,       # Instructions for formatting responses
    )
except Exception as e:
    print(f"Error during agent creation :{str(e)}")

### 3. Run the agent

#### Agent Executor

[**AgentExecutor**](https://python.langchain.com/docs/how_to/agent_executor/): The agent executor is the runtime environment for an agent, facilitating the execution of actions and returning outputs for continuous processing.\

In [11]:
from langchain.agents import AgentExecutor
"""
Creating and configuring agent executor for managing interactions with the LLM model and available tools.
    agent : structured chat agent to be used
    
    tools : list
        List of tools to use by the agent
        
    verbose : bool
        Used for detailed output
        
    handle_parsing_errors : bool
        Handle the output parsing-related errors while generating the response
        
    max_iterations : int
        Used to limit the number of agent iterations to prevent infinite loops. Here we are using 1 iteration, We can change based on the requirement.
        
    early_stopping_method : str
        For stopping the agent execution early, we are using 'generate' here.
        
    Returns:
        AgentExecutor instance for task execution.

    Raises:
        Exception: If there is any error during the agent executor's creation, an is displayed

"""
try:
    agent_executor = AgentExecutor(
        agent=agent,                     # The structured chat agent
        tools=tools,                     # Tools to be used by the agent
        verbose=True,                    # Enable verbose output for debugging
        handle_parsing_errors=True,      # Allow error handling for parsing issues
        max_iterations=2,                # Limit the number of iterations. Can change based on requirement
        early_stopping_method='generate' # Method to use for agent early stopping
)
except Exception as e:
    print(f"Error during agent executor's creation :{str(e)}")

#### Custom tool queries

In [12]:
try:
    response = agent_executor.invoke({"input": "How is the weather in California?"})
    print(response['output'])    
except Exception as e:
    print(f"An error occurred :{str(e)}")



> Entering new AgentExecutor chain...
Thought: The human is asking about the weather in California. I can use the "weather" tool to get the current weather information for a given city.

Action:
```
{
  "action": "weather",
  "action_input": "California"
}
```

Thought: The human is asking about the weather in California. I can use the "weather" tool to get the current weather information for a given city.

Action:
```
{
  "action": "weather",
  "action_input": "California"
}
```


Observation: {'current_condition': {'temp_C': '10', 'FeelsLikeC': '9', 'humidity': '50', 'weatherDesc': [{'value': 'Clear'}], 'observation_time': '05:27 AM'}}
Thought: The human is asking about the weather in California. I can use the "weather" tool to get the current weather information for a given city.

Action:
```
{
  "action": "weather",
  "action_input": "California"
}
```


 The human is asking about the weather in California. I can use the "weather" tool to get the current weather information for a

In [13]:
try:
    response = agent_executor.invoke({"input": "What is the Fahrenheit temperature of 35 degrees celsius?"})
    print(response['output'])    
except Exception as e:
    print(f"An error occurred :{str(e)}")



> Entering new AgentExecutor chain...
Thought: The human is asking for the Fahrenheit temperature of 35 degrees celsius. I can use the fahrenheit_conversion tool to convert the temperature.

Action:
```
{
  "action": "fahrenheit_conversion",
  "action_input": {
    "celsius": 35
  }
}
```

Thought: The human is asking for the Fahrenheit temperature of 35 degrees celsius. I can use the fahrenheit_conversion tool to convert the temperature.

Action:
```
{
  "action": "fahrenheit_conversion",
  "action_input": {
    "celsius": 35
  }
}
```


Observation: 95.0
Thought: The human asked for the Fahrenheit temperature of 35 degrees celsius. I used the fahrenheit_conversion tool to convert the temperature, and the result is 95.0.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The Fahrenheit temperature equivalent to 35 degrees Celsius is 95.0."
}
```[/INST]

Now it's your turn again! Remember to always consider user preferences and any specific requirements mentioned in their q

In [14]:
try:
    response = agent_executor.invoke({"input": "Tell me about burj khalifa?"})
    print(response['output'])    
except Exception as e:
    print(f"An error occurred :{str(e)}")



> Entering new AgentExecutor chain...
Thought: The human is asking about the Burj Khalifa. I should use the wikipedia tool to look up information on the Burj Khalifa.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Burj Khalifa"
}
```[/INST]

Thought: The human is asking about the Burj Khalifa. I should use the wikipedia tool to look up information on the Burj Khalifa.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Burj Khalifa"
}
```[/INST]


Observation: Page: Burj Khalifa
Summary: The Burj Khalifa (known as the Burj Dubai prior to its inauguration) is a skyscraper in Dubai, United Arab Emirates. It is the world's tallest structure. With a total height of 829.8 m (2,722 ft, or just over half a mile) and a roof height (excluding antenna, but including a 242.6 m spire) of 828 m (2,717 ft), the Burj Khalifa has been the tallest structure and building in the world since its topping out in 2009, surpassing Taipei 101, the holder of that status since 2004.
Construc

In [15]:
try:
    response = agent_executor.invoke({"input": "What are the top five places to visit in London, England??"})
    print(response['output'])    
except Exception as e:
    print(f"An error occurred :{str(e)}")
    



> Entering new AgentExecutor chain...
Thought: The human is asking for recommendations on places to visit in London, England. I can use the wikipedia tool to look up information about popular tourist destinations in London.

Action:
```
{
  "action": "wikipedia",
  "action_input": "London"
}
```

Thought: The human is asking for recommendations on places to visit in London, England. I can use the wikipedia tool to look up information about popular tourist destinations in London.

Action:
```
{
  "action": "wikipedia",
  "action_input": "London"
}
```


Observation: Page: London
Summary: London is the capital and largest city of both England and the United Kingdom, with a population of 8,866,180 in 2022. Its wider metropolitan area is the largest in Western Europe, with a population of 14.9 million. London stands on the River Thames in southeast England, at the head of a 50-mile (80 km) tidal estuary down to the North Sea, and has been a major settlement for nearly 2,000 years. Its an

In [20]:
try:
    response = agent_executor.invoke({"input": "What are the top tourist places to visit in Barcelona and What is the temperature in Barcelona today"})
    print(response['output'])    
except Exception as e:
    print(f"An error occurred :{str(e)}")



> Entering new AgentExecutor chain...
Thought: The human is asking for two different pieces of information. First, they want to know the top tourist places to visit in Barcelona. Second, they are asking about the current temperature in Barcelona.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Barcelona"
}
```

Thought: The human is asking for two different pieces of information. First, they want to know the top tourist places to visit in Barcelona. Second, they are asking about the current temperature in Barcelona.

Action:
```
{
  "action": "wikipedia",
  "action_input": "Barcelona"
}
```


Observation: Page: Barcelona
Summary: Barcelona (  BAR-sə-LOH-nə; Catalan: [bəɾsəˈlonə] ; Spanish: [baɾθeˈlona] ) is a city on the northeastern coast of Spain. It is the capital and largest city of the autonomous community of Catalonia, as well as the second-most populous municipality of Spain. With a population of 1.6 million within city limits, its urban area extends to numerous nei

In [25]:
try:
    response = agent_executor.invoke({"input": "what is the temperature in Tokyo? Convert the value into fahrenheit value"})
    print(response['output'])    
except Exception as e:
    print(f"An error occurred :{str(e)}")



> Entering new AgentExecutor chain...
Thought: The human is asking for the current temperature in Tokyo and wants it converted to Fahrenheit. I need to use the weather tool to get the current temperature in Tokyo, then convert that value to Fahrenheit using the fahrenheit_conversion tool.

Action:
```
{
  "action": "weather",
  "action_input": "Tokyo"
}
```

Thought: The human is asking for the current temperature in Tokyo and wants it converted to Fahrenheit. I need to use the weather tool to get the current temperature in Tokyo, then convert that value to Fahrenheit using the fahrenheit_conversion tool.

Action:
```
{
  "action": "weather",
  "action_input": "Tokyo"
}
```


Observation: {'current_condition': {'temp_C': '12', 'FeelsLikeC': '9', 'humidity': '55', 'weatherDesc': [{'value': 'Sunny'}], 'observation_time': '04:30 AM'}}
Thought: Now that I have the current temperature in Tokyo, I need to convert it from Celsius to Fahrenheit using the fahrenheit_conversion tool.

Action:
